Во многих популярных курсах машинного и глубокого обучения вас научат классифицировать собак и кошек, предсказывать цены на недвижимость, покажут еще десятки задач, в которых машинное обучение, вроде как, отлично работает. Но вам расскажут намного меньше (или вообще ничего) о тех случаях, когда ML-модели не работают так, как ожидалось.

Частой проблемой в машинном обучении является неспособность ML-моделей корректно работать на большем разнообразии примеров, чем те, что встречались при обучении. Здесь идет речь не просто о других примерах (например, тестовых), а о других *типах* примеров. Например, сеть обучалась на изображениях коровы, в которых чаще всего корова был на фоне травы, а при тестировании требуется корректное распознавание коровы на любом фоне. Почему ML-модели часто не справляются с такой задачей и что с этим делать - мы рассмотрим далее. Работа над этой проблемой важна не только для решения практических задач, но и в целом для дальнейшего развития ИИ.

~~В большинстве курсов по машинному обучению об этом почему-то не рассказывают. На мой взгляд, начинать ML- и DL-курсы нужно не с простых "hello-world" примеров и не с абстрактной теории, а с конкретных ситуаций, на которых ML-модели *ошибаются*. Это подготовит отличную почву для понимания теории (чтобы хорошо понять теорию, нужно сначала прочувствовать проблемы на практике).~~

Конечно, этим проблемы машинного обучения не ограничиваются, поэтому ~~рассмотрим и другие многочисленные проблемы, такие как сложности с интерпретацией модели, проблемы предвзятости и этики, ресурсоемкости обучения и прочие.~~

## Проблемы обобщения

### Задача обобщения в машинном обучении

Задача машинного обучения заключается в написании алгоритмов, которые автоматически выводят общие закономерности из частных случаев (обучающих примеров). Этот процесс называется обобщением (*generalization*), или индукцией. Часто требуется найти зависимость между исходными данными ($X$) и целевыми данными ($y$) в виде функции $f: X \to y$.

Например, требуется определить по фотографии тип или положение изображенного объекта. Для решения этой задачи можно собрать большое количество размеченных данных (*supervised learning*), можно также использовать ненадежно размеченные данные (*weakly-supervised learning*), например фотографии из Instagram c хештегами, или даже неразмеченные данные (*self-supervised learning*). Но в любом случае мы используем некий набор конкретных примеров.

Понятно, что чем больше доступно примеров, тем выше будет качество полученного решения  (при прочих равных условиях). Больше данных - выше точность. Все логично?

Существуют даже теоремы, доказывающие для различных ML-алгоритмов стремление получаемого решения к идеалу при неограниченном увеличении количества обучающих данных и размера модели (это свойство называется *universal consistency*; например, для нейронных сетей см. [Faragó and Lugosi, 1993]($Strong Universal Consistency of Neural Network Classifiers$)). При доказательстве подобных теорем считается, что данные $X, y$ берутся из некоего фиксированного, но неизвестного распределения $P(X, y)$. Такой подход называется *statistical learning framework*, он излагается почти в любой книге по машинному обучению (например, см. [Deep Learning book]($Deep Learning$), раздел [5.2](https://www.deeplearningbook.org/contents/ml.html), есть также русское издание).

С развитием технологий стали доступны дешевые вычисления и огромные объемы данных, что позволило эффективно решать очень сложные задачи. Например, языковая модель [GPT-3](https://habr.com/ru/post/514698/) ([Brown et al., 2020]($Language Models are Few-Shot Learners$)) от компании OpenAI обучалась на 570 Гб текстов и имеет 175 миллиардов параметров, по [приблизительным подсчетам](https://www.reddit.com/r/MachineLearning/comments/hwfjej/d_the_cost_of_training_gpt3/) обучение стоило миллионы долларов. Примеры текстов, которые генерирует GPT-3, можно найти [здесь](https://twitter.com/raphaelmilliere/status/1289129723310886912), [здесь](https://twitter.com/raphaelmilliere/status/1289135398644580352), [здесь](https://www.gwern.net/GPT-3) и [здесь](https://tech.liga.net/technology/article/ne-v-moih-interesah-vas-unichtojit-neyroset-gpt-3-naehala-na-tsukerberga-eto-konets). В Google обучили сеть с 1.6 триллиона параметров ([Fedus et al., 2021]($Switch Transformers: Scaling to Trillion Parameter Models with Simple and Efficient Sparsity$)), и это далеко не предел: уже [делаются попытки](https://www.zdnet.com/article/cerebras-prepares-for-the-era-of-120-trillion-parameter-neural-networks/) обучать сеть со 120 триллионами параметров, что примерно равно количеству синапсов в [человеческом мозге](https://insh.world/science/elaborate-visualization-human-brain-date/). Более крупные модели и больше обучающих данных - выше точность.

Казалось бы, что может пойти не так?

Рассмотрим несколько примеров, когда простого увеличения объема обучающих данных может оказаться недостаточно. Начнем с проблемы утечки данных, и затем рассмотрим более общую проблему ограниченного разнообразия обучающих данных и ситуации, когда модель лишь "делает вид", что обучается.

### Утечка данных

Утечкой данных называется ситуация, когда существует некий признак, который при обучении содержал больше информации о целевой переменной, чем при последующем применении модели на практике. Утечка данных может возникать в самых разных формах.

**Пример 1.** При диагностике заболеваний по рентгеновским снимкам модель обучается и тестируется на данных, собранных с разных больниц. Рентгеновские аппараты в разных больницах выдают снимки с немного разной тональностью, при этом различия могут быть даже незаметны человеческим глазом. Модель может обучиться определять больницу на основе тональности снимка, а значит и вероятный диагноз (в разных больницах лежат люди с разными диагнозами), при этом вообще не анализируя изображение на снимке. Такая модель покажет отличную точность при тестировании, так как тестовые снимки взяты из той же выборки данных.

**Пример 2.** В задаче диагностики [реальных и фейковых вакансий](https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction) почти все фейковые вакансии относились к Европе. Обученная на этих данных модель может не считать подозрительными любые вакансии из других регионов. При применении на практике такая модель станет бесполезна, как только фейковые вакансии станут размещаться в других регионах. Конечно можно удалить регион из признаков, но тогда модель будет пытаться предсказать регион по другим признакам (скажем, по отдельным словам в описании и требованиях), и проблема сохранится.

**Пример 3.** В той же задаче есть признак "имя компании", и у компаний может быть по нескольку вакансий. Если компания фейковая, то все ее вакансии фейковые. Если мы разделим данные на обучающую и тестовую часть (или на фолды) таким образом, что вакансии одной и той же компании попадут и в обучающую, и в тестовую часть, то модели достаточно будет запомнить имена фейковых компаний для предсказания на тестовой выборке, в результате на тесте мы получим сильно завышенную точность.

<img src="assets/leakage.jpg" width="300" align="center">

Все описанные проблемы нельзя решить просто увеличением количества обучающих данных без изменения подхода к обучению. Здесь можно возразить, что утечка данных не является проблемой самих ML-алгоритмов, поскольку алгоритм и не может знать о том, какой признак нужно учитывать, а какой нет. Но давайте посмотрим на другие похожие примеры.

### Shortcut learning: "right for the wrong reasons"

В начале XX века жила лошадь по имени Умный Ганс, которая якобы умела решать сложные арифметические задачи, постукивая копытом нужное число раз. В итоге оказалось, что лошадь определяет момент, когда нужно прекращать стучать копытом, по выражению лица того, кто задает вопрос. С тех пор имя "умного Ганса" стало нарицательным ([1]($Unmasking Clever Hans Predictors and Assessing What Machines Really Learn$), [2]($When Choosing Plausible Alternatives, Clever Hans can be Clever$)) - оно означает получение решения обходным, нечестным путем, не решая при этом саму задачу в том смысле, какой мы хотим. Такое обходное "решение" внезапно перестает работать, если меняются условия (например, человек, задающий вопрос лошади, сам не знает ответа на него).

Исследователи давно заметили, что многие современные модели глубокого обучения похожи на "умного Ганса". Вспомним хотя бы легендарную [статью](https://habr.com/ru/post/259191/) на Хабре про "леопардовый диван":

> ... Где-то в этот момент в мозг начинает прокрадываться ужасная догадка — а что если вот это различие в текстуре пятен и есть главный критерий, по которому алгоритм отличает три возможных класса распознавания [леопарда, ягуара, гепарда] друг от друга? То есть на самом деле сверточная сеть не обращает внимания на форму изображенного объекта, количество лап, толщину челюсти, особенности позы и все вот эти тонкие различия, которые, как мы было предположили, она умеет понимать — и просто сравнивает картинки как два куска текстуры?

> Это предположение необходимо проверить. Давайте возьмем для проверки простую, бесхитростную картинку, без каких-либо шумов, искажений и прочих факторов, осложняющих жизнь распознаванию. Уверен, эту картинку с первого взгляда легко опознает любой человек. <br> <br> <img src='assets/leopard.jpg' width=250/>

> ... Похоже, что наша догадка насчет текстуры близка к истине. ... Я сделал еще пару проверок (Microsoft, Google) — некоторые из них ведут себя лучше, не подсовывая ягуара, но победить повернутый набок диван не смог никто. Неплохой результат в мире, где уже мелькают заголовки в духе "{Somebody}'s Deep Learning Project Outperforms Humans In Image Recognition".

В самом деле, зачем модели, решающей задачу классификации, выучивать форму и другие комплексные свойства объекта, называемого "леопард", если простое присутствие нужной текстуры на изображении дает почти 100%-ю точность выявления леопарда в обучающей (и тестовой) выборке из ImageNet?

То же самое касается предсказания объекта с учетом фона. Мы не закладывали при обучении никакой информации о том, на основании чего мы хотим получать предсказание: на основании объекта или окружающего его фона. Было даже обнаружено ([Zhu et al., 2016]($Object Recognition with and without Objects$)), что нейронные сети способны с хорошей точностью предсказывать класс объекта, если сам объект вырезан и оставлен только фон.

**Shortcut learning** - явление, когда модели получают верный ответ с помощью неверных в общем случае рассуждений ("right for the wrong reasons"), которые хорошо работают только для обучающего распределения данных. Поскольку обучающая и тестовая выборка обычно берутся из одного распределения, то такие модели могут давать хороую точность и при тестировании.

> Shortcuts are decision rules that perform well on standard benchmarks but fail to transfer to more challenging testing conditions, such as real-world scenarios.

Другие примеры можно найти в замечательной статье Shortcut Learning in Deep Neural Networks ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)), опубликованной в журнале Nature и основанной на обобщении информации из более чем 140 источников.

Смена угла поворота объекта, появление его в необычном (с точки зрения обучающей выборки) окружении, добавление необычного шума может полностью испортить работу модели. На иллюстрации ниже слева показаны пары изображений, одинаково классифицируемые с точки зрения человека, но совершенно разные с точки зрения сверточной нейронной сети. Справа показаны, наоборот, изображения, имеющие одинаковый класс для нейронной сети и разный для человека.

<img src="assets/generalisation.jpg" width="800" align="center">

Ссылки на работы, процитированные на иллюстрации:
- [Wang and Deng, 2018]($Deep Visual Domain Adaptation: A Survey$). Deep Visual Domain Adaptation: A Survey.
- [Szegedy et al., 2013](https://arxiv.org/abs/1312.6199). Intriguing properties of neural networks (см. также [обзор]($Intriguing properties of neural networks$)).
- [Dodge and Karam, 2019]($Human and DNN Classification Performance on Images With Quality Distortions: A Comparative Study$). Human and DNN Classification Performance on Images With Quality Distortions: A Comparative Study.
- [Alcorn et al., 2019]($Strike with a Pose: Neural Networks Are Easily Fooled by Strange Poses of Familiar Objects$). Strike (with) a Pose: Neural Networks Are Easily Fooled by Strange Poses of Familiar Objects.
- [Geirhos et al., 2019]($ImageNet-trained CNNs are biased towards texture; increasing shape bias improves accuracy and robustness$). ImageNet-trained CNNs are biased towards texture; increasing shape bias improves accuracy and robustness (см. также [обзор](https://habr.com/ru/company/ods/blog/453788/) на Хабре).
- [Beery et al., 2018]($Recognition in Terra Incognita$). Recognition in Terra Incognita.
- [Jacobsen et al., 2019]($Excessive Invariance Causes Adversarial Vulnerability$). Excessive Invariance Causes Adversarial Vulnerability.
- [Nguyen et al., 2015]($Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images$). Deep Neural Networks are Easily Fooled: High Confidence Predictions for Unrecognizable Images.
- [Hendrycks et al., 2019]($Natural Adversarial Examples$). Natural Adversarial Examples.
- [Brendel and Bethge, 2019]($Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$). Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet.

В целом исследователями был сделан вывод о том, что многие современные сверточные нейронные сети предпочитают ориентироваться на локальные участки текстур ([Jo and Bengio, 2017]($Measuring the tendency of CNNs to Learn Surface Statistical Regularities$); [Geirhos et al., 2018]($ImageNet-trained CNNs are biased towards texture; increasing shape bias improves accuracy and robustness$)), которые могут относиться как к объекту, так и к фону, и не классифицируют объект на основании его общей формы ([Baker et al., 2018]($Deep convolutional networks do not classify based on global object shape$)).

### Shortcut learning в языковых моделях

Та же проблема есть и в языковых моделях. Например, модель BERT ([Devlin et al., 2018]($BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding$)) обучена на таком объеме текстов, который человек не прочитает за всю жизнь, но при этом она в основном выучивает поверхностные, стереотипные ассоциации слов друг с другом (word co-occurence), и вряд ли хорошо понимает смысл текста.

> Some of BERT's world knowledge success comes from learning stereotypical associations, e. g. a person with an Italian-sounding name is predicted to be Italian, even when it is incorrect. ([Rogers et al., 2020]($A Primer in BERTology: What we know about how BERT works$))

Попробуем протестировать одни из наиболее популярных на сегодняшний день моделей на основе BERT: RoBERTa ([Liu et al., 2019]($RoBERTa: A Robustly Optimized BERT Pretraining Approach$)) и ALBERT ([Lan et al., 2019]($ALBERT: A Lite BERT for Self-supervised Learning of Language Representations$)), взяв для тестирования их крупные варианты из репозитория HuggingFace: [roberta-large](https://huggingface.co/roberta-large) и [albert-xxlarge-v2](https://huggingface.co/albert-xxlarge-v2). Как и BERT, эти модели обучались на задаче предсказания закрытых маской слов в тексте, называемой masked language model objective, или cloze task (в дополнение, ALBERT использует также sentence-order prediction loss). Проверим их работу на различных примерах. Вы можете сами повторить эксперимент, используя окно "Hosted inference API" по приведенным выше ссылкам.

<img src="assets/mlm.jpg" width="600" align="center">

Как видим, модели неплохо запоминают простые факты, но плохо понимают логику повествования. Если логика повествования противоречит статистике совместной встречаемости слов, то модели обычно предпочитают статистику.

Вероятно, модель ALBERT обучалась на фразах о том, что киты питаются крошечными существами, и поэтому решила, что киты - это крошечные животные. В последних двух примерах модель ALBERT, скорее всего, работает так: если "approaches", то "discuss", если "approach", то "propose" - потому что она часто видела такие сочетания. Слово "existing" при этом ее не смущает. Скорее всего, такая модель вообще нисколько "не понимает" общего значения слов "propose" и "discuss".

Вполне вероятно, что будь модель в 100 раз крупнее (и обучаясь в 100 раз дольше и на большем объеме данных), она не совершила бы таких очевидных ошибок. Но даже эти модели обучались очень долго и на огромных данных, чему качество их работы (на нетипичных примерах) явно не соответствует - это наводит на мысль о неоптимальности подхода в целом.

Проблема shortcut learning присутствует и в тех моделях, которые специально дообучались для [лидербордов](https://gluebenchmark.com/leaderboard) GLUE ([Wang et al., 2018]($GLUE: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding$)) и SuperGLUE ([Wang et al., 2019]($SuperGLUE: A Stickier Benchmark for General-Purpose Language Understanding Systems$)). Эти бенчмарки нацелены на измерение уровня "понимания естественного языка" моделями машинного обучения. SuperGLUE содержит 8 задач, на некоторых из которых уже побит уровень человека (имеется в виду средний уровень человека, отличающегося от разметчика, т. к. разметка тоже делалась людьми).

Но действительно ли уровень человека побит? Как выясняется, высокая метрика качества в SuperGLUE достигается в основном с помощью shortcut learning, при этом модели эксплуатируют систематические проблемы в разметке датасета для предсказания ответа "обходным путем" ([Gururangan et al., 2018]($Annotation Artifacts in Natural Language Inference Data$); [Du et al., 2021]($Towards Interpreting and Mitigating Shortcut Learning Behavior of NLU Models$)).

### Уровни обобщения моделей машинного обобщения

Почти любой датасет имеет ограниченное разнообразие и не покрывает всех ситуаций, в которых желательна корректная работа модели. Особенно это проявляется в случае сложных данных, таких как изображения, тексты и звукозаписи. В данных могут существовать "паразитные корреляции" (*spurious correlations*), позволяющие с хорошей точностью предсказывать ответ только на данной выборке без "комплексного" понимания изображения или текста.

Это как раз и есть рассмотренные выше предсказания на основе фона, отдельных текстурных пятен или отдельных рядом стоящих слов. Датасеты, в которых паразитные корреляции явно выражены, называются *biased* (предвзятыми). Пожалуй, что все датасеты в задачах CV и NLP в той или иной степени предвзяты.

В статье о shortcut learning ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)) авторы рассматривают несколько уровней обобщения, которые могут достигать модели машинного обучения:

<img src="assets/taxonomy.jpg" width="340" align="center">

**Uninformative features**. Сеть использует признаки, которые не позволяют эффективно предсказывать ответ даже на обучающей выборке, например, случайно инициализированные веса нейронной сети.

**Overfitting features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ на обучающей выборке, но не на всем распределении, из которого получена эта выборка. Под "распределением" здесь понимается вероятностное совместное распределение $P(x, y)$, из которого взяты данные (более подробно см. также [этот обзор](https://habr.com/ru/company/ods/blog/599573/), раздел "распределение данных").

**Shortcut features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ на распределении данных $P(x, y)$, из которого взята обучающая (и, как правило, тестовая) выборка. Поскольку выборка является набором независимых примеров, используется термин *independent and identically distributed ([i. i. d.](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D1%8B%D0%B5_%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2%D0%BE_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D0%B5_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD%D1%8B))*. Способность алгоритма с хорошей точностью работать на некоем фиксированном распределении данных авторы называют *i. i. d. обобщением*. Как мы видели выше, для этого вовсе не обязательно уметь решать саму поставленную задачу в общем виде, часто можно использовать "обходные пути" или утечки данных.

**Intended features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ в общем случае. Такие признаки будут хорошо работать и вне обучающего распределения данных, когда "обходные пути" закрыты (например, объект находится на необычном фоне, в необычной позиции, имеет необычную текстуру и пр.).

Ситуация, когда распределение данных отличается при обучении и применении называется сдвигом распределения данных (*distributional shift*). Иными словами, данные, на которых модель будет применяться, среднестатистически отличаются от тех, на которых модель обучалась и тестировалась. В частности, может идти речь о появлении новых типов примеров или изменении соотношения количества примеров разных типов. Это происходит из-за того, что обучающие данные часто недостаточно разнообразны и не покрывают все те типы примеров, на которых желательна корректная работа модели (либо покрывают их в неправильном соотношении).

Хорошая работа модели в условиях сдвига данных называется *out-of-distribution обобщением*. Такой уровень обобщения часто недоступен современным ML-моделям. [Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$) так описывают проблему:

> Generalizing beyond one's experiences – a hallmark of human intelligence from infancy – remains a formidable challenge for modern AI.

Еще один типичный пример - системы распознавания лиц. В обучающем датасете, даже очень большом, может ни быть ни одного изображения человеческого лица, на которое нанесен некий рисунок. Модель, обученная на таком датасете, научится быть инвариантной к положению лица в кадре и его выражению, и почти безошибочно идентифицировать человека. Однако модель может перестать корректно работать в случае, если надеты необычные [очки](https://www.theverge.com/2016/11/3/13507542/facial-recognition-glasses-trick-impersonate-fool) или на лицо нанесен [рисунок](https://www.rbth.com/science_and_tech/2017/07/22/hiding-from-artificial-intelligence-in-the-age-of-total-surveillance_808692), потому что *изображений такого типа не встречалось при обучении* (либо они встречались чрезвычайно редко и практически не оказали влияния на обучение).

В результате люди придумывают все новые и новые способы обманывать системы распознавания лиц, не пряча при этом полностью лицо. Модель не встречалась с такими изображениями при обучении и распознает лица неправильно, даже если человек на месте модели легко опознал бы знакомое лицо.

<img src="assets/facerec.jpg" width="550" align="center">

Конечно, распознавание лиц может не всегда использоваться во благо обществу, но здесь речь идет об общей проблеме систем машинного обучения. Связь этих примеров с shortcut learning неочевидна, но явно видна связь с недостатком out-of-distribution обобщения, которое можно рассматривать как одну из метрик "интеллектуальности" модели.

### Проблема неконкретизации в ML-задачах

В данном разделе мы рассмотрим еще одну работу, изучающую поведение ML-моделей за пределами обучающего распределения данных.

Для каждой задачи может существовать бесконечное количество разных обученных моделей с разными весами и вычислительными архитектурами, дающих примерно одинаковую точность на *обучающей выборке*. Это известная и центральная проблема машинного обучения. Но эта же проблема существует и в другом варианте: для каждой задачи может существовать бесконечное количество разных обученных моделей, дающих примерно одинаковую точность на *обучающем распределении данных*, из которого взята тестовая выборка.

В работе [D'Amour et al., 2020]($Underspecification Presents Challenges for Credibility in Modern Machine Learning$) эту проблему называют проблемой неконкретизации ML-задачи (*underspecification of ML-pipeline*). Например, если задачей является обучить систему распознавания изображений с помощью сверточной сети, то может существовать огромное множество разных сверточных сетей с разной архитектурой и инициализацией весов, дающих примерно одинаковую точность на тестовой выборке. При смене условий (выходе за пределы обучающего распределения) эти модели могут начать работать сильно по-разному.

В одном из экспериментов авторы обучили на ImageNet 50 сверточных нейронных сетей ResNet-50 ([He et al., 2015]($Deep Residual Learning for Image Recognition$)). На валидационной части ImageNet все сети давали примерно одинаковую точность 75.9% ± 0.1% (указано одно стандартное отклонение). Затем авторы протестировали сети на датасете ImageNet-С ([Hendrycks and Dietterich, 2019]($Benchmarking Neural Network Robustness to Common Corruptions and Perturbations$)), содержащем изображения с различными шумами и искажениями (out-of-distrubiton данные с точки зрения ImageNet). В результате на изображениях с пикселизацией сети давали точность 19.7% ± 2.4%, на изображениях с уменьшенной или увеличенной контрастностью сети давали точность 9.1% ± 0.8% и так далее. Аналогичная ситуация наблюдалась на сетях ResNet-101x3 BiT ([Kolesnikov et al., 2019]($Big Transfer BiT: General Visual Representation Learning$)).

<img src="assets/underspec.jpg" width="700" align="center">

Видно, что точность моделей не только стала намного хуже, но и увеличился разброс значений точности среди моделей (2.4% против 0.1%). Более того, отклонения от средней точности на разных типах искажений почти не коррелируют. Это означает, что если модель $A$ справляется лучше модели $B$ на изображениях с пикселизацией, то это не означает, что она будет лучше справляться на изображениях с измененной контракстностью.

Подобный эксперимент авторы повторили и с другими данными: на медицинских изображениях, где использовались изображения и 5 камер (первые 4 камеры - обучающее распределение, 5-я камера - out-of-distrubiton данные), и получили аналогичный результат. Таким образом, *поведение моделей, имеющих одинаковую точность на тестовой выборке из обучающего распределения, может быть существенно разным за пределами этого распределения*.

Наконец, авторы сравнивают две предобученные модели BERT (с разной инициализацией весов при предобучении), которые затем файн-тюнились на датасете STS-B для оценки сходства двух текстов. Авторы тестируют работу этих моделей, рассчитывая следующее значение:

$F(X) = \text{sim}(\text{"a woman is walking", "a } X \text{ is walking"}) - \text{sim}(\text{"a man is walking", "a } X \text{ is walking"})$

Где вместо "X" ставилось название профессии. На графике ниже точками отмечены профессии, по вертикали - значение $F(X)$, по горизонтали - процент женщин, занятых в этой профессии (из данных статистики). Как можно видеть, одна из моделей сильнее полагается на стереотипную ассоциацию профессии с полом при рассчете сходства двух предложений, чем другая.

<img src="assets/stsb.jpg" width="250" align="center">

А ведь в описанных выше экспериментах речь шла о моделях лишь с разной инициализацией весов, что уж говорить о моделях с разной архитектурой? Авторы предполагают, что не только архитектура, но и, скажем, оптимизатор и стратегия управления learning rate может сильно влиять на свойства модели (помимо общей точности на обучающем распределении) и точность работы полученной модели на тех или иных примерах за пределами обучающего распределения.

### Выводы

1. ML-модели часто полагаются на "обходные" способы получения ответа (shortcuts), вызванные недостаточным разнообразием обучающего распределения данных и наличия в нем паразитных корреляций (это напоминает утечку данных). Такая модель лишь имитирует решение задачи, и поэтому может перестать корректно работать, если условия изменятся.
2. Следовательно, модели часто показывают низкую точность работы на таких данных, вероятность встретить которые в обучающем датасете была невелика: например, диван леопардовой расцветки, лицо с нанесенной краской, объекты в необычном окружении или текст, в котором используются нестереотипные словосочетания.
3. Модели, показывающие одинаковую среднюю точность при тестировании, могут работать существенно по-разному, что особенно проявляется за пределами распределения данных, на котором модели обучались и тестировались.

Здесь еще можно вспомнить платформу Kaggle для проведения ML-соревнований. Часто считают, что рейтинг на Kaggle не полностью отражает уровень ML-специалиста из-за того, что реальный процесс ML-разработки содержит намного больше шагов, чем просто обучения модели. Однако при этом считается, что Kaggle хорошо развивает умение обучать эффективные по метрикам качества ML-модели. Исходя из рассмотренного выше можно усомниться и в этом факте. На Kaggle качество моделей как правило оценивается на том же распределении, из которого взята обучающая выборка, поэтому проблемы shortcut learning и устойчивость модели к сдвигу данных не играет особой роли. Кроме того, утечки данных (как разновидность сдвига данных) в реальной работе вредны, а на Kaggle скорее полезны.

## Способы решения проблем обобщения

Закончив раздел "кто виноват", перейдем к разделу "что делать". Вопрос в том, как изгнать "умного Ганса" из ML-моделей и обучить их в полной мере решать поставленную задачу. Рассмотрим сначала наиболее прямолинейные способы, и закончим исследовательскими гипотезами и обзором работ, посвященных разработкам новых архитектур и методов обучения.

### Стресс-тесты для диагностики работы модели

Авторы статей о shortcut learning ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)) и неконкретизации ([D'Amour et al., 2020]($Underspecification Presents Challenges for Credibility in Modern Machine Learning$)) сходятся в том, что необходимо проверять не только оценивать общую точность модели на тестовой выборке, но и оценивать точность на отдельных типах примеров. В частности, авторы предлагают следующие типы тестов:

1. **Stratified evaluations.** Разбиение тестовой выборки на типы (например, по значению какого-то признака, по классу) и оценка точности модели отдельно на каждом типе примеров. Процитирую пример из [другого обзора](https://habr.com/ru/company/ods/blog/599573/):

> Пусть модель тестировалась на датасете, в котором 80% изображений были высокого качества (HQ), а применяться будет в условиях, когда, наоборот, 80% изображений будут низкого качества (LQ). Пусть мы сравниваем две модели: на HQ-изображениях точность первой модели лучше, чем второй, а на LQ-изображениях, наоборот, точность второй модели лучше, чем первой. Если при тестировании большая часть изображений были HQ, то мы сделаем вывод, что первая модель лучше, тогда как на самом деле лучше была бы вторая.

2. **Shifted evaluations.** Тестирование модели на всех типах данных, на которых желательна ее хорошая работа. Например, на зашумленных данных, на фотографиях с разных времен года и так далее.

3. **Contrastive evaluations.** Вместо рассчета средней метрики качества модели по какой-либо выборке, мы изучаем изменение метрики качества при изменении выборки (или одного примера). Например, поворачиваем фотографии на один и тот же угол и сравниваем метрику качества. В задачах обработки естественного языка [Ribeiro et al., 2020]($Beyond Accuracy: Behavioral Testing of NLP Models with CheckList$) предлагают целый ряд проверок, которые стоит проделать на обученных моделях.

Перечисленные способы помогут вывить проблемы в функционировании модели. После этого соответствующие примеры можно добавить в обучающие данные (или увеличить их процентное соотношение). Тем самым out-of-distribution данные мы превращаем в in-distribution данные и повышаем качество работы модели на этих данных.

*Дополнение.* Если модель работает с табличными данными и в процессе ее мониторинга доступны размеченные данные, то можно использовать [SHAP loss values](https://habr.com/ru/company/ods/blog/599573/) ([Lundberg et al., 2019]($Explainable AI for Trees: From Local Explanations to Global Understanding$)) для диагностики ситуаций, когда отдельные значения отдельных признаков начинают ухудшать точность модели. Выявив такую ситуацию, нужно определить ее причину: это может быть утечка данных, ошибка в коде, сдвиг в распределении исходных признаков $P(X)$ или в условном распределении целевого признака $P(Y|X)$.

### Еще больше данных?

Предложенный выше способ (добавить в датасет недостающие типы данных) может быть не применим, если задачей является обучение модели общего назначения, например, предобучение языковой модели, сети для распознавания изображений, сети для распознавания речи или же мультимодальной сети, наподобие CLIP ([Radford et al., 2021]($Learning Transferable Visual Models From Natural Language Supervision$)). В таких случаях скрытых bias'ов в данных и возможностей для shortcut learning неисчислимое множество. Обязательно ли в этом случае искать новые архитектуры или способы обучения?

От себя могу предположить, что, как ни странно, еще большее увеличение объема обучающих данных может быть решением. С увеличением объема данных почти всегда растет разнообразие данных (умозрительно можно предположить, что если объем растет линейно, то разнообразие растет логарифмически). А ведь разнообразие данных - это именно то, что нужно? Возможно, с какого-то момента, когда loss уже нельзя будет еще сильнее понизить простым запоминанием поверхностной статистики (т. е. с помощью shortcut'ов), процесс оптимизации будет стагнировать и будет вынужден начать искать другие пути понижения loss, в том числе через понимание сложных аспектов логики повествования или структуры изображения, что и означает систематическую генерализацию.

Конечно эта лишь гипотеза. Но на мой взгляд важно иметь в виду, что зависимость качества обучения от размера модели и объема данных может быть необычной и непредсказуемой. Например, в [Nakkiran et al., 2019]($Deep Double Descent: Where Bigger Models and More Data Hurt$) (double descent) была показана необычная зависимость точности на валидации от размера модели, а в [Power et al., 2022]($Grokking: Generalization Beyond Overfitting on Small Algorithmic Datasets$) (grokking) от длительности обучения.

<img src="assets/unexpected.jpg" width="700" align="center">

### Более крупные или более эффективные модели?

Сейчас глубокое обучение в основном идет по пути экстенсивного развития, при котором улучшения достигаются увеличением объема обучающих данных и размера моделей ("stack more layers"). Разработке принципиально иных архитектур и способов обучения уделяется мало внимания и финансирования, что и понятно: в таких исследованиях нет никакой гарании успеха и получения работающего решения в обозримые сроки. Но как мы увидели выше, простое увеличение объема данных не всегда дает решение, обладающее желаемым уровнем обобщения. Об этом говорят авторы работы, посвященной графовым сетям ([Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$)):

> The question of how to build artificial systems which exhibit combinatorial generalization has been at the heart of AI since its origins, and was central to many structuredapproaches, including logic, grammars, classic planning, graphical models, causal reasoning, Bayesian nonparametrics, and probabilistic programming  ([Chomsky, 1957]($Syntactic structures$); [Nilsson and Fikes, 1970]($STRIPS: A New Approach to the Application of Theorem Proving to Problem Solving$); [Pearl, 1986]($Fusion, Propagation, and Structuring in Belief Networks$), [2009]($Causality: Models, Reasoning and Inference$); [Russell and Norvig, 2009]($Artificial Intelligence: A Modern Approach$); [Hjort et al., 2010]($Bayesian Nonparametrics$); [Goodman et al., 2012]($Church: a language for generative models$); [Ghahramani, 2015]($Probabilistic machine learning and artificial intelligence$)). ... A key reason why structured approaches were so vital to machine learning in previous eras was, in part, because *data and computing resources were expensive*, and the improved sample complexity afforded by structured approaches’ strong inductive biases was very valuable.

> In contrast with past approaches in AI, modern deep learning methods ([LeCun et al., 2015]($Deep Learning Nature Journal$);
[Schmidhuber, 2015]($Deep Learning in Neural Networks: An Overview$); [Goodfellow et al., 2016]($Deep Learning$)) often follow an "end-to-end" design philosophy which emphasizes minimal *a priori* representational and computational assumptions, and seeks to avoid explicit structure and "hand-engineering". This emphasis hasfit well with — and has perhaps been affirmed by — the current abundance of *cheap data and cheap computing resources*, which make trading off sample efficiency for more flexible learning a rational choice. ...

> Despite deep learning’s successes, however, important critiques ([Marcus, 2001]($The algebraic mind$); [Shalev-Shwartz et al., 2017]($Failures of Gradient-Based Deep Learning$); [Lake et al., 2017]($Building Machines That Learn and Think Like People$); [Lake and Baroni, 2018]($Generalization without systematicity: On the compositional skills of sequence-to-sequence recurrent networks$); [Marcus, 2018a]($Deep Learning: A Critical Appraisal$), [2018b]($Innateness, AlphaZero, and Artificial Intelligence$); [Pearl, 2018]($Theoretical Impediments to Machine Learning With Seven Sparks from the Causal Revolution$); [Yuille and Liu, 2018]($Deep Nets: What have they ever done for Vision?$)) have highlighted key challenges it faces in *complex language and scene understanding, reasoning about structured data, transferring learning beyond the training conditions, and learning from small amounts of experience*. These challenges demand combinatorial generalization, and so it is perhaps not surprising that an approach which eschews compositionality and explicit structure struggles to meet them.

Во введении мы говорили о языковой модели GPT-3, имеющей огромный размер и генерирующей реалистичные тексты. Вот, что о ней [думает](https://www.facebook.com/yann.lecun/posts/10157253205637143) Ян Лекун:

> Some people have completely unrealistic expectations about what large-scale language models such as GPT-3 can do. ... GPT-3 doesn't have any knowledge of how the world actually works. *It only appears to have some level of background knowledge, to the extent that this knowledge is present in the statistics of text. But this knowledge is very shallow and disconnected from the underlying reality. ...trying to build intelligent machines by scaling up language models is like building a high-altitude airplanes to go to the moon*. You might beat altitude records, but going to the moon will require a completely different approach.

<img src="assets/gpt3world.jpg" width="550" align="center">

Рост результатов на различных бенчмарках, например SuperGLUE, тоже в основном обусловлен ростом размера моделей и количества обучающих данных. Но обучающие данные все же не бесконечны. Современные языковые модели и так обучаются почти на всех данных, какие удалось собрать со всего интернета.

В какой-то момент, скорее всего, потребуется прекратить рост размера моделей, объема датасетов и сосредоточиться на разработке совершенно иных подходов, стандартизировав наборы обучающих данных (как предлагает [Linzen, 2020]($How Can We Accelerate Progress Towards Human-like Linguistic Generalization?$)) для лучшего сравнения эффективности подходов.

<img src="assets/nostack.jpg" width="300" align="center">

### Модификации способов обучения языковых моделей

Современные языковые модели предобучаются, например, на следующих задачах:

1. Предсказание токенов (т. е. слов или их частей), закрытых маской (masked language modeling, BERT, [Devlin et al., 2018]($BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding$)). Иногда используют модификацию этой задачи, в которой закрываются маской словосочетания, представляющие собой целостные сущности (knowledge masking, ERNIE, [Sun et al., 2019]($ERNIE: Enhanced Representation through Knowledge Integration$)).

2. Предсказания следующего токена в тексте (generative pre-training, GPT, [Radford et al., 2018]($Improving Language Understanding by Generative Pre-Training$)). При этом на архитектуру модели накладывается ограничение: $i$-й входной токен не должен влиять на выходные токены с индексами меньше $i$.

3. Предсказание отдельных токенов в тексте, полученном конкатенацией предложения из энциклопедии и информации из графа знаний (universal knowledge-text prediction, ERNIE 3.0, [Sun et al., 2021]($ERNIE 3.0: Large-scale Knowledge Enhanced Pre-training for Language Understanding and Generation$)). Кроме этой задачи, модель ERNIE 3.0 также обучается на задачах generative pre-training и knowledge masked language modeling.

4. Наборы sequence-to-sequence задач, генерируемых в self-supervised режиме, в том числе аналоги задачи masked language modeling (T5, [Raffel et al., 2019]($Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer$), см. figure 2, table 3).

Чтобы лучше понять свойства этих задач, можно попробовать порешать их самостоятельно. Я сделал [Colab-ноутбук](https://colab.research.google.com/drive/17p6DD8kcpkp-nBRcIFfD7jAmAxmN7rSN?usp=sharing) для генерации случайных обучающих примеров в задаче masked language modeling (MLM). Из примеров можно видеть следующее:

1. Существенная часть замаскированных токенов определяется тривиально

2. Во многих случаях предскать замаскированные токены исходя из контекста невозможно, а значит для решения задачи на большом корпусе текста нужно запоминать множество различных фактов: сюжеты литературных произведений, историю, политику и так далее, а также множество статистики совместной встречаемости слов в тексте (shortcuts). Даже выучив множество фактов и статистики, решить задачу MLM можно далеко не всегда.

3. Лишь изредка возможно предсказать замаскированный токен на основании контекста и логики, и чаще всего эта логика достаточно простая. Исчезающе редки примеры, где для предсказания нужна нетривиальная логика, которую нельзя имитировать запоминанием фактов и статистики.

Следовательно, приоритетом для снижения loss на задаче MLM является запоминание фактов и статистики. Способность рассуждать и делать выводы (отображая слова в некую модель мира и обратно, как это делает человек) для снижения loss является лишь второстепенным фактором, при этом выучить эту способность должно быть намного сложнее, чем запомнить факты.

Получается, что мы обучаем сеть на такой задаче, которую нельзя хорошо решить, обладая одной лишь логикой и пониманием смысла общеупотребимых слов, но можно неплохо решить с помощью запоминания огромного объема фактов и shortcut'ов. Обученная таким способом модель действительно может оказаться способной генерировать правдоподобные тексты, но лишь "вспоминая", что когда-то видела что-то похожее. Таким образом, модель долго и старательно обучается, но вовсе не тому, чего мы от нее хотим.

<img src="assets/holmes.jpg" width="650" align="center">

Отсюда можно сделать вывод о том, что обучение сетей, которые действительно понимают смысл текста в том смысле, в каком его понимает человек (вместо использования shortcut'ов для имитации понимания текста) будет затруднительным, если принципиально не менять задачу предобучения.

### Архитектура моделей и структура данных

Модель эффективно обобщается, если ее структура соответствует структуре моделируемых данных. По этой причине на выборке, сгенерированной случайно инициализированной нейронной сетью, хорошо обобщается другая нейронная сеть ([Gorishniy et al., 2021]($Revisiting Deep Learning Models for Tabular Data$)), на выборке, сгенерированной случайным решающим деревом, хорошо обобщаются другие деревья, сверточные сети хорошо обобщаются на текстурах и составленных из них изображениях ([Ulyanov et al., 2017]($Deep Image Prior$)), а AlphaFold 2 хорошо обобщается на трехмерных структурах белков ([Jumper et al., 2021]($Highly accurate protein structure prediction with AlphaFold$)). Какова же структура текстов и изображений? Можно назвать два основных уровня:

*Уровень 1. Низкоуровневая локальная структура*. В изображениях это набор локальных признаков (текстуры и границы), в текстах это буквы и морфология слов. Также сюда можно отнести уровень звукового/визуального восприятия текста и интонацию.

*Уровень 2. Высокоуровневая иерархическая разреженная структура*. В текстах эта структура начинается с синтаксиса, следующим уровнем явлется связь между предложениями, абзацами и более крупными частями текста. В изображениях обычно тоже есть некая иерархическая структура, соответствующая объектам и их частям в трехмерном пространстве на разном расстоянии. При этом между элементами иерархической структуры имеется разреженный граф смысловых и причинно-следственных взаимосвязей. Например, рассуждая о человеке на фотографии, мы никак не будем учитывать цвет стоящего неподалеку автомобиля. Разреженность помогает не обращать внимание на паразитные корреляции и избегать shortcut learning'а.

Кроме того, когда человек воспринимает фотографию, иерархическая структура изображения отображается в модель мира, и человек сознательно или подсознательно выводы о том, логично ли изображенное на фотографии, в каком контексте и зачем оно сделано и каково развитие событий в момент съемки. С текстом все в целом аналогично.

Судя по всему, для исправления проблемы shortcut learning модель должна реализовывать именно такой способ обработки изображения или текста, то есть уметь представлять входные данные (изображение или текст) как иерархическую комбинацию составляющих частей с разреженными взаимосвязями, что соответствует уровню 2, и быть способной корректно интерпретировать те же части в необычных комбинациях. По сравнению с абстрактным свойством out-of-distribution обобщения, это более конкретное желаемое свойство называют *систематическим, или комбинаторным обобщением* ([Bahdanau et al., 2018]($Systematic Generalization: What Is Required and Can It Be Learned?$)). Достижение такого уровня обобщения остается открытой проблемой машинного обучения.

<iframe width="560" height="315" src="https://www.youtube.com/embed/T3sxeTgT4qc" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

> If you think about child learning, for example, their world is changing over time, their body is changing over time, and so we need systems that are going to be able to handle those changes and do things like continual learning, life-long learning and so on. This has been a long-standing goal for machine learning, but I think we haven't yet built the solutions to this. And one of the crucial elements in order to be successful in this ... is introducing more forms of compositionality. It means being able to learn from some finite sets of combinations about a much larger set of combinations. (Yoshua Bengio, NeurIPS 2019)

Здесь часто вспоминают [символьные подходы](https://en.wikipedia.org/wiki/Symbolic_artificial_intelligence), с которых и начинались исследования ИИ в XX веке. По структуре такие системы, кажется, обладают всем необходимым: они могут работать с объектами и их частями, то есть с интерпретируемыми промежуточными данными, работать иерархически, реализовывать сложные цепочки рассужедний, а вычисления внутри них могут затрагивать лишь немногие объекты и взаимосвязи, что означает разреженность. Непонятно лишь одно: как их обучать, чтобы они могли быть сложными и достаточно гибкими одновременно.

Например, пусть некая подсистема решает, какую из операций из набора $\{F_1, F_2, F_3\}$ применить к данным $x$. Для этого она выдает вероятности $P(F_1), P(F_2), P(F_3)$ и выбирает операцию с наибольшей вероятностью. Допустим, на текущем примере мы выбрали операцию $A = \text{argmax}_i P(F_i)$, и в итоге получили значение функции потерь $L$. Производные $\partial L / \partial P(F_i)$ равны нулю, поэтому такую систему не получится обучить обычным градиентным спуском. Можно попробовать рассчитать ответ, используя по очереди все $F_i$, и снизить вероятности $P(F_i)$ для тех $F_i$, на которых loss получился выше, чем на других. Но если граф принятия решений содержит не одну, а много операций принятия решения ($\text{argmax}$), тогда придется перебирать очень много разных вариантов последовательностей решений. Аналогичная проблема возникает, если мы выбираем не операцию, а один или несколько элементов среди доступных данных (*hard attention*).

TODO

~~Проблемы трансформера, иерархический трансформер http://www.generalized.ru/25.01.22~~

~~GAN без "дальнодействия"~~

~~Обучение на синтаксическом дереве, построенном другой сетью~~

~~Неявные графовые сети, разреженные комбинаторные представления~~

~~Насколько хорошо современные модели соответствуют этой структуре?~~

~~Сверточные нейронные сети хорошо способны обучаться низкоуровневой локальной структуре: текстурам и границам. Кажется, что архитектура сверточных сетей способна кодировать и высокоуровневую иерархическую структуру: часто можно встретить утверждение, что признаки в разных слоях кодируют "объекты разных масштабов": например, части лица и лица целиком.~~

~~Как было показано, ..., сверточные сети действительно могут выучивать иерархические представления в том смысле, что при визуализации активаций промежуточных слоев (тем или иным способом) в них проглядываются знакомые объекты.~~

~~Кажется, что эти иллюстрации говорят сами за себя. Но почему же тогда современные сверточные сети подвержены тем проблемам, которые мы видели в предыдущем разделе?~~

- [Lake and Baroni, 2017]($Generalization without systematicity: On the compositional skills of sequence-to-sequence recurrent networks$). Generalization without systematicity.
- [Ross et al., 2017]($Right for the Right Reasons: Training Differentiable Models by Constraining their Explanations$). Training Differentiable Models by Constraining their Explanations.
- [Bengio, 2017]($The Consciousness Prior$). The Consciousness Prior.
- [Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$). Relational inductive biases, deep learning, and graph networks.
- [Bahdanau et al., 2018]($Systematic Generalization: What Is Required and Can It Be Learned?$). Systematic Generalization: What Is Required and Can It Be Learned?
- [Schölkopf, 2019]($Causality for Machine Learning$). Causality for Machine Learning.
- [Goyal and Bengio, 2020]($Inductive Biases for Deep Learning of Higher-Level Cognition$). Inductive Biases for Deep Learning of Higher-Level Cognition.
- [Hinton, 2021]($How to represent part-whole hierarchies in a neural network$). How to represent part-whole hierarchies in a neural network.

~~In particular, by averaging across many randomly perturbed predictors from a single
modeling pipline, deep ensembles have been shown to be effective tools for detecting out-of-distribution
inputs, and correspondingly for tamping down the confidence of predictions for such inputs ([D'Amour et al., 2020]($Underspecification Presents Challenges for Credibility in Modern Machine Learning$)) - ссылаются на https://arxiv.org/abs/1906.02530~~

## Заключение

